# HW4: Self-Attention

Speaker Classification using Transformer with Keras 3.0

Optimized version with improved architecture and training pipeline

Original dataset is **Voxceleb1**. We randomly select 600 speakers from Voxceleb1.Then preprocess the raw waveforms into mel-spectrograms.

In [1]:
# HomeworkID
HOMEWORK_ID = "hw4"
# Res Directory
RES_DIR = f"res/{HOMEWORK_ID}/"
# Output Directory
OUTPUT_DIR = f"output/{HOMEWORK_ID}/"
# Random Seed
RANDOM_SEED = 42

In [2]:
# Standard libraries
import os
import gc
import random
import numpy as np
import csv
import json
from pathlib import Path

# Machine learning libraries
import torch
import keras
import matplotlib.pyplot as plt

c:\ProgramData\anaconda3\envs\pytorch\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
# Utilities
def get_device():
    ''' Returns the available device: GPU if available, else CPU '''
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_random_seed(seed_value=RANDOM_SEED):
    ''' Sets the random seed for reproducibility '''
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
        # torch.backends.cudnn.deterministic = True # 强制CuDNN使用确定性算法
        # torch.backends.cudnn.benchmark = False # 关闭CuDNN的自动优化功能

def plot_learning_curve(loss_record: dict, title: str):
    """Plots the learning curve"""
    total_epochs = len(loss_record["loss"])
    x_train = np.arange(1, total_epochs + 1)
    
    if len(loss_record["val_loss"]) > 0:
        x_val = x_train[:: max(1, len(x_train) // len(loss_record["val_loss"]))]
        x_val = x_val[:len(loss_record["val_loss"])]
    else:
        x_val = []
    
    plt.figure(figsize=(12, 5))
    
    # Loss plot
    plt.subplot(1, 2, 1)
    plt.plot(x_train, loss_record["loss"], c="tab:red", label="Training Loss", linewidth=2)
    if len(x_val) > 0:
        plt.plot(x_val, loss_record["val_loss"], c="tab:blue", label="Validation Loss", linewidth=2)
    plt.xlabel("Epochs", fontsize=12)
    plt.ylabel("Loss", fontsize=12)
    plt.title("Loss Curve", fontsize=14)
    plt.legend()
    plt.grid(alpha=0.3)
    
    # Accuracy plot
    plt.subplot(1, 2, 2)
    if "accuracy" in loss_record:
        plt.plot(x_train, loss_record["accuracy"], c="tab:green", label="Training Accuracy", linewidth=2)
    if "val_accuracy" in loss_record and len(x_val) > 0:
        plt.plot(x_val, loss_record["val_accuracy"], c="tab:orange", label="Validation Accuracy", linewidth=2)
    plt.xlabel("Epochs", fontsize=12)
    plt.ylabel("Accuracy", fontsize=12)
    plt.title("Accuracy Curve", fontsize=14)
    plt.legend()
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"{title.replace(' ', '_')}.png"), dpi=150)
    plt.show()

def clear_memory():
    """清理内存"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [4]:
# GPU Checking
print(f"{'='*40}")
print(f"Using device: {get_device()}")
print(f"PyTorch version: {torch.__version__}")
print(f"Keras Version: {keras.__version__}")
print(f"Current Backend: {keras.backend.backend()}")
print(f"{'='*40}")

# Set random seed for reproducibility
set_random_seed(RANDOM_SEED)
print(f"Random seed set to: {RANDOM_SEED}")
print(f"{'='*40}")

# Make sure OUTPUT_DIR exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

Using device: cuda
PyTorch version: 2.9.1+cu130
Keras Version: 3.13.0
Current Backend: torch
Random seed set to: 42


## Dataset Classes

In [5]:
class SpeakerDataset(torch.utils.data.Dataset):
    """Dataset for speaker classification training"""
    
    def __init__(self, data_dir, segment_len=128):
        self.data_dir = Path(data_dir)
        self.segment_len = segment_len
        
        # Load mappings
        with open(self.data_dir / "mapping.json") as f:
            mapping = json.load(f)
        self.speaker2id = mapping["speaker2id"]
        
        # Load metadata
        with open(self.data_dir / "metadata.json") as f:
            metadata = json.load(f)["speakers"]
        
        self.speaker_num = len(metadata.keys())
        self.data = []
        
        for speaker in metadata.keys():
            for utterance in metadata[speaker]:
                self.data.append([
                    utterance["feature_path"], 
                    self.speaker2id[speaker]
                ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        mel = torch.load(self.data_dir / feat_path)
        
        # Segment or pad
        if len(mel) > self.segment_len:
            start = random.randint(0, len(mel) - self.segment_len)
            mel = mel[start:start + self.segment_len]
        
        mel = torch.FloatTensor(mel)
        speaker = torch.LongTensor([speaker])
        
        return mel, speaker
    
    def get_speaker_number(self):
        return self.speaker_num


class InferenceDataset(torch.utils.data.Dataset):
    """Dataset for inference"""
    
    def __init__(self, data_dir):
        self.data_dir = Path(data_dir)
        
        with open(self.data_dir / "testdata.json") as f:
            metadata = json.load(f)
        
        self.data = metadata["utterances"]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        utterance = self.data[index]
        feat_path = utterance["feature_path"]
        mel = torch.load(self.data_dir / feat_path)
        
        return feat_path, torch.FloatTensor(mel)

## Collate Functions

In [6]:
def collate_batch(batch):
    """Collate function for training"""
    mels, speakers = zip(*batch)
    
    # Pad sequences
    mels = torch.nn.utils.rnn.pad_sequence(
        mels, batch_first=True, padding_value=-20.0
    )
    speakers = torch.cat(speakers)
    
    return mels.numpy(), speakers.numpy()


def inference_collate_batch(batch):
    """Collate function for inference"""
    feat_paths, mels = zip(*batch)
    mels = torch.nn.utils.rnn.pad_sequence(
        mels, batch_first=True, padding_value=-20.0
    )
    return feat_paths, mels.numpy()

## PyTorch DataLoader Adapter for Keras

In [7]:
class TorchDataLoaderAdapter(keras.utils.PyDataset):
    """Adapter to use PyTorch DataLoader with Keras"""
    
    def __init__(self, dataloader, **kwargs):
        super().__init__(**kwargs)
        self.dataloader = dataloader
        self.iterator = None
    
    def __len__(self):
        return len(self.dataloader)
    
    def __getitem__(self, idx):
        if self.iterator is None:
            self.iterator = iter(self.dataloader)
        
        try:
            batch = next(self.iterator)
        except StopIteration:
            self.iterator = iter(self.dataloader)
            batch = next(self.iterator)
        
        return batch

## Model

In [8]:
@keras.saving.register_keras_serializable()
class TransformerSpeakerClassifier(keras.Model):
    """
    Improved Transformer-based Speaker Classifier
    
    Features:
    - Multi-head self-attention
    - Layer normalization
    - Dropout for regularization
    - Statistics pooling (mean + std)
    """
    
    def __init__(
        self, 
        n_mels=40, 
        d_model=256, 
        n_heads=4, 
        n_layers=4,
        ff_dim=1024,
        n_spks=600, 
        dropout=0.1,
        **kwargs
    ):
        super().__init__(**kwargs)
        
        self.n_mels = n_mels
        self.d_model = d_model
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.ff_dim = ff_dim
        self.n_spks = n_spks
        self.dropout_rate = dropout
        
        # Input projection
        self.input_projection = keras.layers.Dense(d_model, name="input_projection")
        self.input_norm = keras.layers.LayerNormalization(epsilon=1e-6)
        self.input_dropout = keras.layers.Dropout(dropout)
        
        # Transformer encoder layers
        self.attention_layers = []
        self.attention_dropout = []
        self.attention_norm = []
        
        self.ffn_layers = []
        self.ffn_dropout = []
        self.ffn_norm = []
        
        for i in range(n_layers):
            # Multi-head attention
            self.attention_layers.append(
                keras.layers.MultiHeadAttention(
                    num_heads=n_heads,
                    key_dim=d_model // n_heads,
                    dropout=dropout,
                    name=f"mha_{i}"
                )
            )
            self.attention_dropout.append(keras.layers.Dropout(dropout))
            self.attention_norm.append(keras.layers.LayerNormalization(epsilon=1e-6))
            
            # Feed-forward network
            self.ffn_layers.append(
                keras.Sequential([
                    keras.layers.Dense(ff_dim, activation='relu'),
                    keras.layers.Dropout(dropout),
                    keras.layers.Dense(d_model),
                ], name=f"ffn_{i}")
            )
            self.ffn_dropout.append(keras.layers.Dropout(dropout))
            self.ffn_norm.append(keras.layers.LayerNormalization(epsilon=1e-6))
        
        # Classification head
        self.classifier = keras.Sequential([
            keras.layers.Dense(d_model, activation='relu'),
            keras.layers.Dropout(dropout),
            keras.layers.Dense(d_model // 2, activation='relu'),
            keras.layers.Dropout(dropout),
            keras.layers.Dense(n_spks)
        ], name="classifier")
    
    def call(self, inputs, training=False):
        """Forward pass"""
        # inputs: (batch_size, seq_len, n_mels)
        x = self.input_projection(inputs)
        x = self.input_norm(x)
        x = self.input_dropout(x, training=training)
        
        # Transformer encoder layers
        for i in range(self.n_layers):
            # Multi-head attention with residual
            attn_output = self.attention_layers[i](x, x, training=training)
            attn_output = self.attention_dropout[i](attn_output, training=training)
            x = self.attention_norm[i](x + attn_output)
            
            # Feed-forward network with residual
            ffn_output = self.ffn_layers[i](x, training=training)
            ffn_output = self.ffn_dropout[i](ffn_output, training=training)
            x = self.ffn_norm[i](x + ffn_output)
        
        # Statistics pooling: concatenate mean and std
        mean = keras.ops.mean(x, axis=1)
        std = keras.ops.std(x, axis=1)
        stats = keras.ops.concatenate([mean, std], axis=-1)
        
        # Classification
        output = self.classifier(stats, training=training)
        
        return output
    
    def get_config(self):
        return {
            "n_mels": self.n_mels,
            "d_model": self.d_model,
            "n_heads": self.n_heads,
            "n_layers": self.n_layers,
            "ff_dim": self.ff_dim,
            "n_spks": self.n_spks,
            "dropout": self.dropout_rate,
        }


## Custom Learning Rate Schedule

In [9]:
class WarmupCosineDecay(keras.optimizers.schedules.LearningRateSchedule):
    """Learning rate schedule with warmup and cosine decay"""
    
    def __init__(self, initial_lr, warmup_steps, total_steps, min_lr=1e-6):
        super().__init__()
        self.initial_lr = initial_lr
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        self.min_lr = min_lr
    
    def __call__(self, step):
        step = keras.ops.cast(step, "float32")
        warmup_steps = keras.ops.cast(self.warmup_steps, "float32")
        total_steps = keras.ops.cast(self.total_steps, "float32")
        
        # Linear warmup
        warmup_lr = self.initial_lr * step / warmup_steps
        
        # Cosine decay
        progress = (step - warmup_steps) / (total_steps - warmup_steps)
        cosine_decay = 0.5 * (1.0 + keras.ops.cos(np.pi * progress))
        decay_lr = self.min_lr + (self.initial_lr - self.min_lr) * cosine_decay
        
        # Choose based on step
        lr = keras.ops.where(step < warmup_steps, warmup_lr, decay_lr)
        
        return lr
    
    def get_config(self):
        return {
            "initial_lr": self.initial_lr,
            "warmup_steps": self.warmup_steps,
            "total_steps": self.total_steps,
            "min_lr": self.min_lr,
        }

## Training Pipeline

In [10]:
def create_datasets(data_dir, batch_size=32, n_workers=8):
    """Create training and validation datasets"""
    
    dataset = SpeakerDataset(data_dir)
    speaker_num = dataset.get_speaker_number()
    
    # Split dataset (90% train, 10% validation)
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size]
    )
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=n_workers,
        collate_fn=collate_batch,
        pin_memory=True,
        drop_last=True,
        persistent_workers=True if n_workers > 0 else False
    )
    
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=n_workers,
        collate_fn=collate_batch,
        pin_memory=True,
        drop_last=False,
        persistent_workers=True if n_workers > 0 else False
    )
    
    return train_loader, val_loader, speaker_num


def train(config):
    """Main training function using Keras fit()"""
    
    print("=" * 80)
    print("Starting Training with Keras 3.0 (PyTorch Backend)")
    print("=" * 80)
    
    # Create dataloaders
    train_loader, val_loader, speaker_num = create_datasets(
        config["data_dir"],
        config["batch_size"],
        config["n_workers"]
    )
    
    print(f"\n📊 Dataset Info:")
    print(f"  Number of speakers: {speaker_num}")
    print(f"  Training samples: {len(train_loader.dataset)}")
    print(f"  Validation samples: {len(val_loader.dataset)}")
    print(f"  Training batches: {len(train_loader)}")
    print(f"  Validation batches: {len(val_loader)}")
    
    # Build model
    model = TransformerSpeakerClassifier(
        n_mels=40,
        d_model=config.get("d_model", 256),
        n_heads=config.get("n_heads", 4),
        n_layers=config.get("n_layers", 4),
        ff_dim=config.get("ff_dim", 1024),
        n_spks=speaker_num,
        dropout=config.get("dropout", 0.1)
    )
    
    # Build model by calling it once
    dummy_input = np.random.randn(1, 128, 40).astype(np.float32)
    _ = model(dummy_input, training=False)
    
    print(f"\n🏗️  Model Architecture:")
    model.summary()
    
    # Learning rate schedule
    lr_schedule = WarmupCosineDecay(
        initial_lr=config["learning_rate"],
        warmup_steps=config["warmup_steps"],
        total_steps=config["total_steps"],
        min_lr=1e-6
    )
    
    # Compile model
    model.compile(
        optimizer=keras.optimizers.AdamW(
            learning_rate=lr_schedule,
            weight_decay=config.get("weight_decay", 0.01)
        ),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name='accuracy'),
            keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top5_accuracy')
        ]
    )
    
    # Callbacks
    callbacks = [
        keras.callbacks.ModelCheckpoint(
            filepath=config["save_path"],
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=config.get("early_stopping_patience", 10),
            mode='max',
            verbose=1,
            restore_best_weights=True
        ),
        keras.callbacks.CSVLogger(
            'training_log.csv',
            append=False
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            verbose=1,
            min_lr=1e-6
        )
    ]
    
    # Calculate epochs
    steps_per_epoch = len(train_loader)
    epochs = (config["total_steps"] + steps_per_epoch - 1) // steps_per_epoch
    
    print(f"\n⚙️  Training Configuration:")
    print(f"  Batch size: {config['batch_size']}")
    print(f"  Learning rate: {config['learning_rate']}")
    print(f"  Warmup steps: {config['warmup_steps']}")
    print(f"  Total steps: {config['total_steps']}")
    print(f"  Steps per epoch: {steps_per_epoch}")
    print(f"  Total epochs: {epochs}")
    
    print("\n" + "=" * 80)
    print("🚀 Training Started...")
    print("=" * 80 + "\n")
    
    # Train using Keras fit()
    history = model.fit(
        train_loader,
        validation_data=val_loader,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    
    print("\n" + "=" * 80)
    print("✅ Training Completed!")
    print(f"  Best validation accuracy: {max(history.history['val_accuracy']):.4f}")
    print("=" * 80)
    
    return model, history

## Inference Pipeline

In [11]:
def inference(config):
    """Inference function using Keras predict()"""
    
    print("\n" + "=" * 80)
    print("🔮 Starting Inference")
    print("=" * 80)
    
    # Load mapping
    with open(Path(config["data_dir"]) / "mapping.json") as f:
        mapping = json.load(f)
    
    speaker_num = len(mapping["id2speaker"])
    
    # Load model
    print(f"\n📂 Loading model from {config['model_path']}...")
    model = keras.saving.load_model(config["model_path"])
    
    # Create dataset
    dataset = InferenceDataset(config["data_dir"])
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=config.get("inference_batch_size", 64),
        shuffle=False,
        num_workers=config["n_workers"],
        collate_fn=inference_collate_batch,
        pin_memory=True
    )
    
    print(f"  Number of test samples: {len(dataset)}")
    print(f"  Inference batches: {len(dataloader)}")
    
    # Collect all data
    all_feat_paths = []
    all_mels = []
    
    print("\n📦 Loading test data...")
    for feat_paths, mels in dataloader:
        all_feat_paths.extend(feat_paths)
        all_mels.append(mels)
    
    all_mels = np.concatenate(all_mels, axis=0)
    
    # Predict using Keras predict()
    print("\n🎯 Running inference...")
    logits = model.predict(all_mels, batch_size=config.get("inference_batch_size", 64), verbose=1)
    preds = np.argmax(logits, axis=-1)
    
    # Prepare results
    print("\n💾 Preparing submission file...")
    results = [["Id", "Category"]]
    for feat_path, pred in zip(all_feat_paths, preds):
        results.append([feat_path, mapping["id2speaker"][str(pred)]])
    
    # Save results
    with open(config["output_path"], 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(results)
    
    print(f"\n✅ Results saved to {config['output_path']}")
    print("=" * 80)

    return logits

## Training

In [12]:
config = {
    # Data
    "data_dir": RES_DIR,
    
    # Model architecture
    "d_model": 256,
    "n_heads": 4,
    "n_layers": 4,
    "ff_dim": 1024,
    "dropout": 0.1,
    
    # Training
    "batch_size": 32,
    "learning_rate": 1e-3,
    "weight_decay": 0.01,
    "warmup_steps": 1000,
    "total_steps": 70000,
    "early_stopping_patience": 10,
    
    # System
    "n_workers": 8,
    
    # Paths
    "save_path": OUTPUT_DIR + "best_model_keras.keras",
    "model_path": OUTPUT_DIR + "best_model_keras.keras",
    "output_path": OUTPUT_DIR + "submission.csv",
    
    # Inference
    "inference_batch_size": 64,
}

In [ ]:
print("\n" + "=" * 80)
print("🎵 Speaker Classification with Keras 3.0 + PyTorch Backend")
print("=" * 80)
print("\nPHASE 1: Training")
print("=" * 80)
model, history = train(config)


🎵 Speaker Classification with Keras 3.0 + PyTorch Backend

PHASE 1: Training
Starting Training with Keras 3.0 (PyTorch Backend)

📊 Dataset Info:
  Number of speakers: 600
  Training samples: 62494
  Validation samples: 6944
  Training batches: 1952
  Validation batches: 217

🏗️  Model Architecture:


Model: "transformer_speaker_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_projection (Dense)        │ (1, 128, 256)          │        10,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (1, 128, 256)          │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mha_0 (MultiHeadAttention)      │ (1, 128, 256)          │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (1, 128, 256)          │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ffn_0 (Sequential)              │ (1, 128, 256)          │       525,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ (1, 128, 256)          │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mha_1 (MultiHeadAttention)      │ (1, 128, 256)          │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_3           │ (1, 128, 256)          │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ffn_1 (Sequential)              │ (1, 128, 256)          │       525,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_4           │ (1, 128, 256)          │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mha_2 (MultiHeadAttention)      │ (1, 128, 256)          │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_5           │ (1, 128, 256)          │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ffn_2 (Sequential)              │ (1, 128, 256)          │       525,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_6           │ (1, 128, 256)          │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 3,411,672 (13.01 MB)

 Trainable params: 3,411,672 (13.01 MB)

 Non-trainable params: 0 (0.00 B)


⚙️  Training Configuration:
  Batch size: 32
  Learning rate: 0.001
  Warmup steps: 1000
  Total steps: 70000
  Steps per epoch: 1952
  Total epochs: 36

🚀 Training Started...



In [ ]:
plot_learning_curve(history.history, title="Speaker_Classification_Training_Curve")

## Inference

In [ ]:
print("\n" + "=" * 80)
print("PHASE 2: Inference")
print("=" * 80)
predictions = inference(config)

In [ ]:
# ===== 预测统计 =====
max_confidences = np.max(predictions, axis=1)
predicted_labels = np.argmax(predictions, axis=1)

print(f"\n预测置信度统计:")
print(f"  平均置信度: {np.mean(max_confidences):.4f}")
print(f"  中位数置信度: {np.median(max_confidences):.4f}")
print(f"  最小置信度: {np.min(max_confidences):.4f}")
print(f"  最大置信度: {np.max(max_confidences):.4f}")

print(f"\n预测标签分布:")
label_counts = np.bincount(predicted_labels, minlength=11)
for i, count in enumerate(label_counts):
    print(f"  类别 {i}: {count} ({count/len(predictions)*100:.1f}%)")

# ===== 可视化 =====
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 置信度分布
axes[0].hist(max_confidences, bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(np.mean(max_confidences), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: {np.mean(max_confidences):.3f}')
axes[0].set_xlabel("Prediction Confidence", fontsize=12)
axes[0].set_ylabel("Frequency", fontsize=12)
axes[0].set_title("Distribution of Prediction Confidences", fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 标签分布
axes[1].bar(range(11), label_counts, edgecolor='black', alpha=0.7)
axes[1].set_xlabel("Class Label", fontsize=12)
axes[1].set_ylabel("Count", fontsize=12)
axes[1].set_title("Distribution of Predicted Labels", fontsize=14)
axes[1].set_xticks(range(11))
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "prediction_analysis.png"), dpi=150)
plt.show()

print("\n" + "="*60)
print("✓ 训练和预测完成！")
print("="*60)

In [ ]:
print("\n" + "=" * 80)
print("🎉 All Done!")
print("=" * 80 + "\n")